## Natural Language to SQL Proof of Concept

A simple proof of concept that allows a non-programmer to write a program in natural language and have it converted to SQL Code.

#### What is natural language?
A natural language, as described when [Dictionary.com](https://www.dictionary.com/browse/natural-language) states, "a language that has developed and evolved naturally, through use by human beings, as opposed to an invented or constructed language, as a computer programming language". Simply put, a natural language is a language that is used by humans to communicate with each other. The setences that you are reading right now are written in natural language.

#### What is SQL?
SQL, or Structured Query Language, is a "Structured Query Language (SQL) is a standardized programming language that is used to manage relational databases and perform various operations on the data in them" [Techtarget.com](https://www.techtarget.com/searchdatamanagement/definition/SQL#:~:text=Structured%20Query%20Language%20(SQL)%20is,on%20the%20data%20in%20them.).

#### The difference between natural language and SQL
Natural language is used to communicate with other humans. SQL is used to communicate with a database. The two languages are very different. For example, in natural language, you can say:
    
    "I want to buy a car, but my budget is no more than $10000"

In SQL, you would say:

    "SELECT * FROM cars WHERE price < 10000"
    
The two sentences are very different, but they both mean the same thing. An engineer who understands both the natural language and SQL can easily convert the two back and forward. If a non-technical tells them they need a report for all sales within the last 30 days, the engineer can easily convert that to SQL and get the data. 

What we want to do is show that not only can an engineer do it, but GPT can as well.


#### The Imports

We will need a number of different python libraries to make this work. We will need to import the following libraries:

In [8]:
# Installed Package Imports
import pandas as pd
import openai
import dotenv
import sqlalchemy

In [2]:
# Python Imports
import os

#### Safeguarding our API Key

We can easily obfuscate the API Key that we are using by storing it in a environment variables file, or a .env file. We can then import the .env file into our notebook and use the API Key without having to worry about it being exposed.

The .env will have an entry that looks like this:

OPENAI=Our-API-Key-is-Stored-Here

This is imported using a package called Dotenv. 

In [5]:
# API Key is Stored in a .env File, please see the .env.example file for more information
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAPI")

#### Data Exploration

First we must import and explore our data just to get an idea of what we are working with

In [6]:
# Read in the sample data 
df = pd.read_csv("data/sales_data_sample.csv")

In [7]:
# View the first 5 rows of the data
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


#### Database Setup

As we want to perform SQL queries we need to store our CSV data in a database. We can do this using SQL Alchemy, using the module called create_engine. We will create a database called "temp_db" that is stored in memory. We then push our data into the database in a table called "sales".

After that, we can perform queries against the database using the text module that we import from SQL Alchemy.

In [9]:
from sqlalchemy import create_engine
from sqlalchemy import text 

In [12]:
# First we are going to create a temporary database in memory
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [15]:
# Now we push the Pandas DF to the database that we can perform SQL queries on. As we set echo to true, we get to see the SQL query that is being executed while creating the table
data = df.to_sql('sales', con=temp_db, if_exists='replace')

2023-03-09 09:39:11,333 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2023-03-09 09:39:11,334 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-09 09:39:11,336 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2023-03-09 09:39:11,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-09 09:39:11,339 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-03-09 09:39:11,340 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-09 09:39:11,343 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("sales")
2023-03-09 09:39:11,344 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-09 09:39:11,349 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-03-09 09:39:11,350 INFO sqlalchemy.engine.Engine [raw sql] ('sales',)
2023-03-09 09:39:11,354 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("sales")
2023-03-

In [16]:
# Now we can perform SQL queries on the data
with temp_db.connect() as con:
    rs = con.execute(text("SELECT * FROM sales LIMIT 5"))
    for row in rs:
        print(row)

2023-03-09 09:39:33,003 INFO sqlalchemy.engine.Engine SELECT * FROM sales LIMIT 5
2023-03-09 09:39:33,004 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()
(0, 10107, 30, 95.7, 2, 2871.0, '2/24/2003 0:00', 'Shipped', 1, 2, 2003, 'Motorcycles', 95, 'S10_1678', 'Land of Toys Inc.', '2125557818', '897 Long Airport Avenue', None, 'NYC', 'NY', '10022', 'USA', None, 'Yu', 'Kwai', 'Small')
(1, 10121, 34, 81.35, 5, 2765.9, '5/7/2003 0:00', 'Shipped', 2, 5, 2003, 'Motorcycles', 95, 'S10_1678', 'Reims Collectables', '26.47.1555', "59 rue de l'Abbaye", None, 'Reims', None, '51100', 'France', 'EMEA', 'Henriot', 'Paul', 'Small')
(2, 10134, 41, 94.74, 2, 3884.34, '7/1/2003 0:00', 'Shipped', 3, 7, 2003, 'Motorcycles', 95, 'S10_1678', 'Lyon Souveniers', '+33 1 46 62 7555', '27 rue du Colonel Pierre Avia', None, 'Paris', None, '75508', 'France', 'EMEA', 'Da Cunha', 'Daniel', 'Medium')
(3, 10145, 45, 83.26, 6, 3746.7, '8/25/2003 0:00', 'Shipped', 3, 8, 2003, 'Motorcycles', 95, 'S10_1678', 'Toys4G